In [92]:
##Libraries##

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import insert
import pyodbc
import numpy as np

In [112]:
##Connection to cmv_uat database##

server = 'tcp:mycmvserver.database.windows.net' 
database = 'cmvdb_uat' 
username = 'ecervantes@cmvlatam.com' 
password = 'Columbia87/' 


connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:mycmvserver.database.windows.net,1433;Database=cmvdb_uat;Uid='+username+';Pwd='+password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive')
cursor = connection.cursor()

In [114]:
ncliente = 116451  ##Blotter + last 5 trades ### 
query = "SELECT * FROM [dbo].[fxblotter.t]"
# Read the SQL query into a DataFrame
fxblotter = pd.read_sql(query, connection, index_col='tradeid', parse_dates=True)
fxblotter.sort_index(inplace=True)
fxblotter['tradedate'] = pd.to_datetime(fxblotter['tradedate'])
fxblotter['valuedate'] = pd.to_datetime(fxblotter['valuedate'])
fxblottert = fxblotter.loc[fxblotter['clienteid'] == ncliente]

query = "SELECT * FROM [dbo].[cstats]"
# Read the SQL query into a DataFrame
cstatsblotter = pd.read_sql(query, connection, index_col='tradeid', parse_dates=True)
cstatsblotter.sort_index(inplace=True)
cstatsblotter['tradedate'] = pd.to_datetime(fxblotter['tradedate'])
cstatsblotter['valuedate'] = pd.to_datetime(fxblotter['valuedate'])
cstatsblottert = cstatsblotter.loc[cstatsblotter['clienteid'] == ncliente]
cstatsblottert.tail(5)
##fxblottert.head(5)

,descliente,tradedate,side,px_sp,px_cover,monex_pxlast,monex_pp,ejecutivo,comccy,montocom,venccy,montoven,valuedate,dealer,clienteid
tradeid,,,,,,,,,,,,,,,
202105170000615,AISA INVERSIONES ENERGETICAS S.A.,2021-05-17,Bid,616.00,617.03,617.03,617.03,ECER,USD,2500.0,CRC,1540000.0,2021-05-17,Esteban Cervantes,116451
202105310000449,AISA INVERSIONES ENERGETICAS S.A.,2021-05-31,Bid,618.75,620.20,620.20,620.20,ECER,USD,8500.0,CRC,5259375.0,2021-05-31,Esteban Cervantes,116451
202106010000419,AISA INVERSIONES ENERGETICAS S.A.,2021-06-01,Bid,619.75,619.81,619.81,619.81,ECER,USD,2500.0,CRC,1549375.0,2021-06-01,Esteban Cervantes,116451
202106170000102,AISA INVERSIONES ENERGETICAS S.A.,2021-06-17,Bid,618.00,618.25,619.10,619.05,ECER,USD,3600.0,CRC,2224800.0,2021-06-17,Esteban Cervantes,116451
202106240000211,AISA INVERSIONES ENERGETICAS S.A.,2021-06-24,Bid,619.82,619.50,619.61,619.88,ECER,USD,11000.0,CRC,6818020.0,2021-06-24,Esteban Cervantes,116451


In [115]:
### C-Stats

#forma 1
#filtro por cliente
#sumo todos los colones y todos los dolares, y calculo los dolares a tasa de cover 
bidfxblottert = fxblottert.loc[fxblotter['comccyid'] == 1 ]
askfxblottert = fxblottert.loc[fxblotter['venccyid'] == 1 ] 
#forma 2 
#diff px_sp y px_cover
spreadbidmpp =  bidfxblottert['monex_pp'] - fxblottert['px_sp'] 
spreadaskmpp =  askfxblottert['px_sp'] - fxblottert['monex_pp'] 
bidcount = fxblottert.loc[fxblotter['comccyid'] == 1, 'montocom'].count()
askcount = fxblottert.loc[fxblotter['venccyid'] == 1, 'montoven'].count()
spreadbidpxc =  bidfxblottert['px_cover'] - fxblottert['px_sp'] 
spreadaskpxc =  askfxblottert['px_sp'] - fxblottert['px_cover']
spreadbidpxl =  bidfxblottert['monex_pxlast'] - fxblottert['px_sp'] 
spreadaskpxl =  askfxblottert['px_sp'] - fxblottert['monex_pxlast']
bidmontoavg =  bidfxblottert.loc[fxblotter['comccyid'] == 1, 'montocom'].mean()
askmontoavg = askfxblottert.loc[fxblotter['venccyid'] == 1, 'montoven'].mean( )
bidsum = fxblottert.loc[fxblotter['comccyid'] == 1, 'montocom'].sum()
asksum = fxblottert.loc[fxblotter['venccyid'] == 1, 'montoven'].sum()
##blendedspread = 


print1 ="Bid Spread to Monex P.P :  {:,} CRC"
print(print1.format(round(spreadbidmpp.mean(),3)))
print1 ="Bid Spread to Cover P.P :  {:,} CRC"
print(print1.format(round(spreadbidpxc.mean(),3)))
print1 ="Bid Spread to Monex last traded PX :  {:,} CRC"
print(print1.format(round(spreadbidpxl.mean(),3)))
print1 ="Bid monto promedio por transaccion :  {:,} USD"
print(print1.format(round(bidmontoavg,3)))
print1 ="Bid total millage :  {:,} USD"
print(print1.format(bidsum))
print1 ="Bids traded :  {:,} "
print(print1.format(bidcount))
print("                                         ")
print("************************************************************************************")
print("                                         ")
print1 ="Ask Spread to Monex P.P :  {:,} CRC"
print(print1.format(round(spreadaskmpp.mean(),3)))
print1 ="Ask Spread to Cover P.P :  {:,} CRC"
print(print1.format(round(spreadaskpxc.mean(),3)))
print1 ="Ask Spread to Monex last traded PX :  {:,} CRC"
print(print1.format(round(spreadaskpxl.mean(),3)))
print1 ="Bid monto promedio por transaccion :  {:,} USD"
print(print1.format(round(askmontoavg,3)))
print1 ="Ask total millage :  {:,} USD"
print(print1.format(asksum))
print1 ="Asks :  {:,}"
print(print1.format(askcount))
print("                                         ")
print("************************************************************************************")
print("                                         ")
print(fxblottert['tradedate'].dt.day_name().value_counts())
##Agregar Histograma para este ultimo print


Bid Spread to Monex P.P :  0.761 CRC
Bid Spread to Cover P.P :  0.614 CRC
Bid Spread to Monex last traded PX :  0.734 CRC
Bid monto promedio por transaccion :  7,662.5 USD
Bid total millage :  61,300.0 USD
Bids traded :  8 
                                         
************************************************************************************
                                         
Ask Spread to Monex P.P :  nan CRC
Ask Spread to Cover P.P :  nan CRC
Ask Spread to Monex last traded PX :  nan CRC
Bid monto promedio por transaccion :  nan USD
Ask total millage :  0.0 USD
Asks :  0
                                         
************************************************************************************
                                         
Thursday    3
Friday      2
Monday      2
Tuesday     1
Name: tradedate, dtype: int64


In [ ]:
## Print Report 
##all trades counted by day of the week 
##better blotter data frame for sales purposes
Aggregate flow analytics report 



